# Basic Simulation Tutorial

This notebook introduces the core concepts of Scene2Sim through hands-on examples.

## Learning Objectives
- Load and inspect scenarios from JSON annotations
- Run basic simulations with safety metric computation
- Visualize results and export data for analysis
- Understand the annotation-native simulation paradigm

In [ ]:
# Setup
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Scene2Sim imports
from Scene2Sim import load_scenario, ADSimulator
from Scene2Sim.core.scene import AgentType
from Scene2Sim.metrics.safety import SafetyMetrics

# Configure visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Part 1: Loading and Inspecting Scenarios

In [ ]:
# Load a sample scenario
scene = load_scenario("scenarios/urban_intersection.json", "intersection_001")

print(f"Scenario: {scene.id}")
print(f"Duration: {scene.duration}s")
print(f"Agents: {len(scene.agents)}")
print(f"Road width: {scene.road.width}m")
print(f"Lanes: {getattr(scene.road, 'n_ego_lanes', 0)} ego, {getattr(scene.road, 'n_opposite_lanes', 0)} opposite")

In [ ]:
# Inspect agents
for agent_id, agent in scene.agents.items():
    print(f"\nAgent: {agent_id}")
    print(f"  Type: {str(agent.agent_type)}")
    print(f"  Size: {agent.length}m x {agent.width}m")
    print(f"  Trajectory: {len(agent.trajectory.waypoints)} waypoints")
    
    if agent.trajectory.waypoints:
        first_wp = agent.trajectory.waypoints[0]
        last_wp = agent.trajectory.waypoints[-1]
        print(f"  Start: t={first_wp.t:.1f}s, pos=({first_wp.x:.1f}, {first_wp.y:.1f}), v={first_wp.v:.1f}m/s")
        print(f"  End:   t={last_wp.t:.1f}s, pos=({last_wp.x:.1f}, {last_wp.y:.1f}), v={last_wp.v:.1f}m/s")

## Part 2: Running Your First Simulation

In [ ]:
# Create simulator with 20 FPS (dt=0.05)
sim = ADSimulator(
    scene, 
    dt=0.05,
    enable_collisions=True,
    enable_metrics=True,
    random_seed=42
)

print(f"Simulator configured:")
print(f"  Timestep: {sim.dt}s ({1/sim.dt:.0f} FPS)")
print(f"  Collision detection: {sim.enable_collisions}")
print(f"  Safety metrics: {sim.enable_metrics}")

In [ ]:
# Run simulation
print("Running simulation...")
log = sim.run(headless=True)

print(f"\nSimulation completed!")
print(f"  Wall-clock time: {log.duration:.3f}s")
print(f"  Simulated time: {log.states[-1].time:.1f}s")
print(f"  Speedup: {log.states[-1].time / log.duration:.0f}x real-time")

## Part 3: Analyzing Safety Metrics

In [ ]:
# Display safety metrics
metrics = log.metrics

print("=== SAFETY ANALYSIS ===")
print(f"Overall Safety: {'SAFE' if metrics['is_safe'] else ' UNSAFE'}")
print(f"Collision-free: {'YES' if metrics['is_collision_free'] else ' NO'}")
print(f"Collision Events: {metrics['n_collisions']}")
print(f"TTC Events (<3s): {metrics['n_ttc_events']}")
print(f"Minimum TTC: {metrics['min_ttc']:.2f}s")
print(f"Mean TTC: {metrics['mean_ttc']:.2f}s")
print(f"Minimum Distance: {metrics['min_distance_overall']:.2f}m")
print(f"Mean Min Distance: {metrics['mean_min_distance']:.2f}m")

In [ ]:
# Convert to DataFrame
df = log.to_dataframe()
print(f"Simulation data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head()

## Part 4: Visualization

In [ ]:
# Plot agent trajectories
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

for agent_id in df['agent_id'].unique():
    agent_data = df[df['agent_id'] == agent_id]
    axes[0,0].plot(agent_data['x'], agent_data['y'], label=agent_id, linewidth=2, marker='o', markersize=2)
axes[0,0].set_xlabel('X Position (m)')
axes[0,0].set_ylabel('Y Position (m)')
axes[0,0].set_title('Agent Trajectories')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)
axes[0,0].axis('equal')

# Speed profiles
for agent_id in df['agent_id'].unique():
    agent_data = df[df['agent_id'] == agent_id]
    axes[0,1].plot(agent_data['time'], agent_data['v'], label=agent_id, linewidth=2)
axes[0,1].set_xlabel('Time (s)')
axes[0,1].set_ylabel('Speed (m/s)')
axes[0,1].set_title('Speed Profiles')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Inter-agent distance
ego_data = df[df['agent_id'] == 'ego']
ped_data = df[df['agent_id'] == 'ped_0']
distances = np.sqrt((ego_data['x'].values - ped_data['x'].values)**2 + (ego_data['y'].values - ped_data['y'].values)**2)
axes[1,0].plot(ego_data['time'], distances, linewidth=2, color='red')
axes[1,0].axhline(y=1.0, color='orange', linestyle='--', alpha=0.7, label='Safety threshold')
axes[1,0].set_xlabel('Time (s)')
axes[1,0].set_ylabel('Distance (m)')
axes[1,0].set_title('Inter-Agent Distance')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Collision timeline
collision_indicators = [1 if bool(state.collisions) else 0 for state in log.states]
times = [state.time for state in log.states]
axes[1,1].fill_between(times, collision_indicators, alpha=0.3, color='red', label='Collision periods')
axes[1,1].set_xlabel('Time (s)')
axes[1,1].set_ylabel('Collision Status')
axes[1,1].set_title('Collision Timeline')
axes[1,1].set_ylim(-0.1, 1.1)
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('basic_simulation_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## Part 5: Comparing Multiple Scenarios

In [ ]:
# Load and compare multiple scenarios
scenario_ids = ['intersection_001', 'intersection_002', 'intersection_003']
results = []

for scenario_id in scenario_ids:
    try:
        scene = load_scenario("scenarios/urban_intersection.json", scenario_id)
        sim = ADSimulator(scene, dt=0.05, random_seed=42)
        log = sim.run(headless=True)
        results.append({
            'scenario_id': scenario_id,
            'n_agents': len(scene.agents),
            'duration': scene.duration,
            'simulation_time': log.duration,
            'is_safe': log.metrics['is_safe'],
            'n_collisions': log.metrics['n_collisions'],
            'min_ttc': log.metrics['min_ttc'],
            'min_distance': log.metrics['min_distance_overall']
        })
        print(f"✅ Completed {scenario_id}")
    except Exception as e:
        print(f"❌ Failed {scenario_id}: {e}")

comparison_df = pd.DataFrame(results)
print(f"\nScenario Comparison:")
comparison_df

In [ ]:
# Visualize scenario comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
safety_counts = comparison_df['is_safe'].value_counts()
axes[0].pie(safety_counts.values, labels=['Unsafe', 'Safe'], autopct='%1.1f%%')
axes[0].set_title('Safety Distribution')

valid_ttc = comparison_df[comparison_df['min_ttc'] < float('inf')]
if not valid_ttc.empty:
    axes[1].bar(valid_ttc['scenario_id'], valid_ttc['min_ttc'])
    axes[1].set_xlabel('Scenario')
    axes[1].set_ylabel('Minimum TTC (s)')
    axes[1].set_title('Minimum Time-to-Collision')
    axes[1].tick_params(axis='x', rotation=45)

speedup = comparison_df['duration'] / comparison_df['simulation_time']
axes[2].bar(comparison_df['scenario_id'], speedup)
axes[2].set_xlabel('Scenario')
axes[2].set_ylabel('Speedup (x real-time)')
axes[2].set_title('Simulation Performance')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Part 6: Export and Further Analysis

In [ ]:
# Export data
df.to_csv('simulation_data.csv', index=False)
comparison_df.to_csv('scenario_comparison.csv', index=False)
print(" Exported data files:")
print("  - simulation_data.csv: Detailed timestep data")
print("  - scenario_comparison.csv: Summary metrics")
print("  - basic_simulation_analysis.png: Visualization")

In [ ]:
# Summary statistics
print("=== SIMULATION SUMMARY ===")
print(f"Scenarios analyzed: {len(results)}")
print(f"Safe scenarios: {comparison_df['is_safe'].sum()}/{len(comparison_df)}")
print(f"Average simulation speedup: {speedup.mean():.1f}x real-time")

if not valid_ttc.empty:
    print(f"TTC Statistics:")
    print(f"  Mean: {valid_ttc['min_ttc'].mean():.2f}s")
    print(f"  Std:  {valid_ttc['min_ttc'].std():.2f}s")
    print(f"  Min:  {valid_ttc['min_ttc'].min():.2f}s")
    print(f"  Max:  {valid_ttc['min_ttc'].max():.2f}s")

## Key Takeaways

1. **Annotation-Native**: Scene2Sim directly ingests dataset annotations without reconstruction
2. **Fast & Deterministic**: Achieves real-time+ performance with reproducible results
3. **Rich Metrics**: Comprehensive safety evaluation with TTC, collision detection, and distance tracking
4. **Research-Ready**: Easy export to standard formats for further analysis

## Next Steps

- [**Perturbation Analysis**](02_perturbation_analysis.ipynb): Systematic scenario modifications
- [**Advanced Safety Metrics**](03_safety_metrics.ipynb): Deep dive into safety evaluation
-  [**Monte Carlo Studies**](04_monte_carlo.ipynb): Large-scale statistical analysis